# 3. Test model

Testing on lexicons to find best embedding models (requires making full lexicons)

Function to output document score and highlight biased words (1/-1). This function will be referred to by `api.py`. This is urgent TODO, do with pretrained model for now